In [ ]:
import requests
import pandas as pd
# URLs for the 3 cities


In [ ]:
urls = {
    'Riga': 'https://api.open-meteo.com/v1/forecast?latitude=56.94&longitude=24.10&hourly=temperature_2m,relative_humidity_2m',
    'Tallinn': 'https://api.open-meteo.com/v1/forecast?latitude=59.43&longitude=24.75&hourly=temperature_2m,relative_humidity_2m',
    'Vilnius': 'https://api.open-meteo.com/v1/forecast?latitude=54.68&longitude=25.27&hourly=temperature_2m,relative_humidity_2m'
}

In [ ]:

# Function to fetch data and save to CSV
def fetch_and_save_data(city, url):
    response = requests.get(url)
    data = response.json()

    # Extract hourly time and temperature_2m and relative_humidity_2m
    hourly_data = data['hourly']
    df = pd.DataFrame({
        'time': hourly_data['time'],
        'temperature': hourly_data['temperature_2m'],
        'humidity': hourly_data['relative_humidity_2m']
    })

    # Save the dataframe to CSV
    df.to_csv(f'{city}_weather_data.csv', index=False)
    print(f"Data for {city} saved to {city}_weather_data.csv")

# Fetch and save data for each city
for city, url in urls.items():
    fetch_and_save_data(city, url)

Data for Riga saved to Riga_weather_data.csv
Data for Tallinn saved to Tallinn_weather_data.csv
Data for Vilnius saved to Vilnius_weather_data.csv


In [ ]:
import sqlite3
import pandas as pd


In [ ]:


# Load the CSV files (replace with your local file paths)
tallinn_file_path = 'Tallinn_weather_data.csv'
riga_file_path = 'Riga_weather_data.csv'
vilnius_file_path = 'Vilnius_weather_data.csv'

# Load the CSV files into DataFrames
df_tallinn = pd.read_csv(tallinn_file_path)
df_riga = pd.read_csv(riga_file_path)
df_vilnius = pd.read_csv(vilnius_file_path)

# Set up SQLite connection and create a database
conn = sqlite3.connect('weather_data.db')

# Save the DataFrames into the SQLite database with appropriate table names
df_tallinn.to_sql('Tallinn_Weather', conn, if_exists='replace', index=False)
df_riga.to_sql('Riga_Weather', conn, if_exists='replace', index=False)
df_vilnius.to_sql('Vilnius_Weather', conn, if_exists='replace', index=False)

# Verify by listing the tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

# Close the SQLite connection
conn.close()

# Display the list of tables to confirm
print(tables)

              name
0  Tallinn_Weather
1     Riga_Weather
2  Vilnius_Weather


In [ ]:
import sqlite3
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Step 1: Connect to the SQLite database
conn = sqlite3.connect('weather_data.db')

# Step 2: Load the data from each city's table into a DataFrame
df_tallinn = pd.read_sql_query("SELECT * FROM Tallinn_Weather", conn)
df_riga = pd.read_sql_query("SELECT * FROM Riga_Weather", conn)
df_vilnius = pd.read_sql_query("SELECT * FROM Vilnius_Weather", conn)

# Close the connection
conn.close()

# Step 3: Convert the 'time' column to datetime if it exists in the data
df_tallinn['time'] = pd.to_datetime(df_tallinn['time'])
df_riga['time'] = pd.to_datetime(df_riga['time'])
df_vilnius['time'] = pd.to_datetime(df_vilnius['time'])

# Step 4: Create the interactive Plotly plot
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces for each city
for city, df, color in [('Tallinn', df_tallinn, 'blue'), ('Riga', df_riga, 'green'), ('Vilnius', df_vilnius, 'red')]:
    fig.add_trace(
        go.Scatter(x=df['time'], y=df['temperature'], mode='lines', name=f"{city} Temperature", line=dict(color=color)),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=df['time'], y=df['humidity'], mode='lines', name=f"{city} Humidity", line=dict(dash='dot', color=color)),
        secondary_y=True
    )

# Step 5: Update layout with dropdowns for city selection and dual y-axes
fig.update_layout(
    title="Temperature and Humidity Comparison by City",
    xaxis_title="Time",
    yaxis_title="Temperature (°C)",
    yaxis2_title="Humidity (%)",
    updatemenus=[{
        'buttons': [
            {'label': 'All Cities', 'method': 'update', 'args': [{'visible': [True] * 6}, {'title': 'Weather Data for All Cities'}]},
            {'label': 'Tallinn', 'method': 'update', 'args': [{'visible': [True, True, False, False, False, False]}, {'title': 'Weather Data for Tallinn'}]},
            {'label': 'Riga', 'method': 'update', 'args': [{'visible': [False, False, True, True, False, False]}, {'title': 'Weather Data for Riga'}]},
            {'label': 'Vilnius', 'method': 'update', 'args': [{'visible': [False, False, False, False, True, True]}, {'title': 'Weather Data for Vilnius'}]},
        ],
        'direction': 'down',
        'showactive': True,
    }]
)

# Step 6: Add interactive date dropdowns (range selectors)
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1 Day", step="day", stepmode="backward"),
                dict(count=7, label="1 Week", step="day", stepmode="backward"),
                dict(count=1, label="1 Month", step="month", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),  # Enables the range slider below the graph
        type="date"
    )
)

# Step 7: Customize axes and grid
fig.update_xaxes(showgrid=True)
fig.update_yaxes(title_text="Temperature (°C)", secondary_y=False)
fig.update_yaxes(title_text="Humidity (%)", secondary_y=True)

# Step 8: Display the interactive plot
fig.show()

/usr/local/lib/python3.10/dist-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

